In [10]:
from pathlib import Path
import pandas as pd
import json
from collections import Counter
import numpy as np
import datetime as dt
import string
from nltk.stem import WordNetLemmatizer
  


terms = []
file_list = [
   'dictionaries/affect_unique.txt',
   'dictionaries/moral_unique.txt'
]
for file_name in file_list:
    print(file_name)
    with open(file_name) as fb:
        for line in fb.readlines():
            terms.append(line.replace('*', '').replace('\n', '' ))

#print(terms)

stopwords = [u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', 
    u"she's", u'her', u'hers', u'herself', u'it', u"it's", u'its', u'itself', u'they', u'them', 
    u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', 
    u'that', u"that'll", u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be',
    u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing',
    u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until',
    u'while', u'of', u'at', 'in', 'to', 'for', 'you', 'no', 'there', 'i', '&amp', "you're", "you've"]


dfs = []
for path in Path('data').rglob('*.jsonl'):
    with open("data/" + path.name) as fb:
        df = pd.read_json(fb.read(), lines=True)
        dfs.append(df)
df = pd.concat(dfs)
df.head()

dictionaries/affect_unique.txt
dictionaries/moral_unique.txt


,type,id,text,created_at,author_id,users,tweets,newest_id,oldest_id,result_count,next_token,geo,withheld
0,retweeted,1.082107e+18,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1.082184e+18,I WANT THE GUN GIMME THE GUN YES YES YES YES YES,2019-01-07 07:54:28+00:00,3.081783e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,retweeted,1.081974e+18,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,retweeted,1.081974e+18,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,retweeted,1.082108e+18,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Word net

Download word net for the stemming to match the moreal and emotional affect

In [28]:
import nltk
nltk.download('wordnet')



[nltk_data] Downloading package wordnet to /home/kslote/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Regression and Predictors

Count the occurences of each moral and affect term on each day. These will be our predictor variables when divided by the total number of tweets on that day. Our $Y$ values will be the number of backgroung checks.


In [31]:
from collections import Counter
import numpy as np
import datetime as dt
import string
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()

day_count = Counter()
vectors = Counter()
df = df[df['text'].notna()]
for i, row in df.iterrows():
    date = row['created_at']
    day = dt.datetime(date.year, date.month, date.day)
    tweet_vector = np.zeros(len(terms))
    words = row['text']
    if not isinstance(words, str):
        continue
    words = words.lower().replace('\n', ' ').split(' ')
    num_words = len(words)
    
    for word in words:
        if word[:4] == 'http':
            continue
        if not word:
            continue
        if word[0] in ['#', '@']:
            continue # Diregard Hashtags, mentions and retweets.
        if word == 'rt':
            continue
        if word in stopwords:
            continue
        if word[-3:] == '…':
            continue
        invalidChars = set(string.punctuation.replace("-", ""))
        if any(char in invalidChars for char in word):
            continue
        elif word.isnumeric():
            continue
        elif "…" in word:
            continue
        elif "\"" in word:
            continue
        else:
            lemma = lemmatizer.lemmatize(word)
            for term in terms:
                if lemma.startswith(term):
                    #print(term, word)
                    i = terms.index(term)
                    tweet_vector[i] += 1

    vectors[day] += tweet_vector
    day_count[day] += 1

In [73]:
import statsmodels.api as sm

Y = []
X = []
df_nics = pd.read_excel(
    'dictionaries/Daily media and background check data.xlsx',
    engine='openpyxl',
    skiprows=[0]
)
for i, row in df_nics.dropna().iterrows():
    #year, month, day,= row['Yea'].split('-')
    date = row['Date']
    day = dt.datetime(
        int(date.year),
        int(date.month),
        int(date.day)
    )
    if day in vectors.keys():
        for day in vectors:
            if date == day:
                Y.append(row['Daily NICS checks'])
                X.append(vectors[day] / day_count[day])

X = sm.add_constant(X)
est = sm.OLS(Y, X)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.917
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     1.441
Date:                Wed, 28 Jul 2021   Prob (F-statistic):            0.00578
Time:                        18:44:01   Log-Likelihood:                -11009.
No. Observations:                1054   AIC:                         2.388e+04
Df Residuals:                     122   BIC:                         2.851e+04
Df Model:                         931                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.138e+04   8656.456      9.402      0.0

# Granger Causality

Calculate the full regression and the lagged regression. Then Calculate the F statistic for the Granger test.

See [here](!https://www.statsmodels.org/stable/_modules/statsmodels/tsa/stattools.html#grangercausalitytests)

In [81]:
from scipy import stats

mxlg = 1
dtaown = sm.add_constant(X[:, 1 : (mxlg + 1)], prepend=False)
dtajoint = sm.add_constant(X[:, 1:], prepend=False)
# Run ols on both models without and with lags of second variable
res2down = sm.OLS(Y, dtaown).fit()
res2djoint = sm.OLS(Y, dtajoint).fit()
F = (res2down.ssr - res2djoint.ssr) / res2djoint.ssr/ mxlg * res2djoint.df_resid
p_value = stats.f.sf(F, mxlg, res2djoint.df_resid),
print(F, p_value)

1329.4075285352653 (1.8854412604125352e-67,)
